# SQL Subqueries

## Introduction

SQL queries can get complex. For example, you might have been a little thrown off by the many to many join in the last lab. There, you had to join four tables. This is just the tip of the iceberg. Depending on how your database is set up, you might have to join subset views of multiple tables. When queries get complex like this, it is often useful to use the concept of subqueries to help break the problem into smaller, more digestible tasks.

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## Our Customer Relationship Management ERD

As a handy reference, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.Connection('data.sqlite')

## Substituting `JOIN` with Subqueries

Let's start with a query of employees from the United States. Using your current knowledge, you could solve this using a join.

In [5]:
q = """
SELECT lastName, firstName, officeCode
FROM employees
JOIN offices
    USING(officeCode)
WHERE country = "USA"
;"""
pd.read_sql(q, conn)

,lastName,firstName,officeCode
0,Bow,Anthony,1
1,Firrelli,Jeff,1
2,Jennings,Leslie,1
3,Murphy,Diane,1
4,Patterson,Mary,1
5,Thompson,Leslie,1
6,Firrelli,Julie,2
7,Patterson,Steve,2
8,Tseng,Foon Yue,3
9,Vanauf,George,3


Another approach would be to use a subquery. Here's what it would look like:

In [7]:
q = """
SELECT lastName, firstName, officeCode
FROM employees
WHERE officeCode IN (SELECT officeCode
                        FROM offices 
                        WHERE country = "USA")
;
"""
pd.read_sql(q, conn)

,lastName,firstName,officeCode
0,Murphy,Diane,1
1,Patterson,Mary,1
2,Firrelli,Jeff,1
3,Bow,Anthony,1
4,Jennings,Leslie,1
5,Thompson,Leslie,1
6,Firrelli,Julie,2
7,Patterson,Steve,2
8,Tseng,Foon Yue,3
9,Vanauf,George,3


There it is, a query within a query! This can be very helpful and also allow you to break down problems into constituent parts. Often queries can be formulated in multiple ways as with the above example. Other times, using a subquery might be essential. For example, what if you wanted to find all of the employees from offices with at least 5 employees?

## Subqueries for Filtering Based on an Aggregation

Think for a minute about how you might write such a query.  

Now that you've had a minute to think it over, you might see some of the challenges with this query. On the one hand, we are looking to filter based on an aggregate condition: the number of employees per office. You know how to do this using the `GROUP BY` and `HAVING` clauses, but the data we wish to retrieve is not aggregate data. We only wish to **filter** based on the aggregate, not retrieve aggregate data. As such, this is a natural place to use a subquery.

In [10]:
q = """
SELECT lastName, firstName, officeCode, employeeNumber
FROM employees
WHERE officeCode IN (
    SELECT officeCode 
    FROM offices 
    JOIN employees
        USING(officeCode)
    GROUP BY 1
    HAVING COUNT(employeeNumber) >= 5
)
;
"""
pd.read_sql(q, conn)

,lastName,firstName,officeCode,employeeNumber
0,Murphy,Diane,1,1002
1,Patterson,Mary,1,1056
2,Firrelli,Jeff,1,1076
3,Bondur,Gerard,4,1102
4,Bow,Anthony,1,1143
5,Jennings,Leslie,1,1165
6,Thompson,Leslie,1,1166
7,Bondur,Loui,4,1337
8,Hernandez,Gerard,4,1370
9,Castillo,Pamela,4,1401


In [11]:
q = """
SELECT *
FROM offices;
"""
pd.read_sql(q, conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
7,27,Boston,+1 977 299 8345,105 Cambridge Street,,MA,USA,02331,NA


You can chain queries like this in many fashions. For example, maybe you want to find the average of individual customers' average payments:

(It might be more interesting to investigate the standard deviation of customer's average payments, but standard deviation is not natively supported in SQLite as it is in other SQL versions like PostgreSQL.)

In [17]:
q = """
SELECT AVG(customerAvgPayment) AS averagePayment
FROM (
    SELECT AVG(amount) AS customerAvgPayment
    FROM payments
    JOIN customers
        USING(customerNumber)
    GROUP BY customerNumber
)
;"""
pd.read_sql(q, conn)

,averagePayment
0,31489.754582


In [14]:
pd.read_sql("""SELECT AVG(amount) AS customerAvgPayment
    FROM payments
    JOIN customers
        USING(customerNumber)
    GROUP BY customerNumber
    """, conn)

,customerAvgPayment
0,7438.120000
1,26726.993333
2,45146.267500
3,38983.226667
4,26056.197500
...,...
93,25908.863333
94,21285.185000
95,14793.075000
96,32770.870000


You can also run subqueries that reference keys with different names between different tables. For example you can use the employee number in the employees table and the matching sales rep employee number in the customers table.

In [19]:
q = """
SELECT lastName, firstName, employeeNumber
FROM employees
WHERE employeeNumber IN (SELECT salesRepEmployeeNumber
                        FROM customers 
                        WHERE country = "USA")
;
"""
pd.read_sql(q, conn)

,lastName,firstName,employeeNumber
0,Jennings,Leslie,1165
1,Thompson,Leslie,1166
2,Firrelli,Julie,1188
3,Patterson,Steve,1216
4,Tseng,Foon Yue,1286
5,Vanauf,George,1323


In [20]:
conn.close()

## Summary

In this lesson, you were briefly introduced to the powerful concept of subqueries and how you can use them to write more complex queries. In the upcoming lab, you'll really start to strengthen your SQL and data wrangling skills by using all of the SQL techniques introduced thus far.